In [1]:
from DB_System.communication import DB_Handle, DataHandle
from BackTesting_System.backtesting import EconomicIndicators, BackTesting, Utils

In [2]:
import pandas as pd
from copy import deepcopy

In [3]:
def make_signal(df):
    df_copy = deepcopy(df)
    object_list = df_copy['종목명'].unique()
    for i in object_list:
        idx = df_copy[df_copy['종목명']==i].index
        
        '''buy 조건 '''
        con1 = (df_copy.loc[idx[0]:idx[-1],'5_mom'].shift(2) < 100) & (df_copy.loc[idx[0]:idx[-1],'5_mom'].shift(2) >= 95)
        con2 = (df_copy.loc[idx[0]:idx[-1],'5_mom'].shift(1) >= 100)&(df_copy.loc[idx[0]:idx[-1],'5_mom'].shift(1) < 110)
        con3 = (df_copy.loc[idx[0]:idx[-1],'5_mom'] >= df_copy.loc[idx[0]:idx[-1],'5_mom'].shift(1)) & (df_copy.loc[idx[0]:idx[-1],'5_mom'] < 110)
        con4 = (df_copy.loc[idx[0]:idx[-1],'누적거래대금'] >= 10000000000)
        df_copy.loc[:, 'buy'] = con1 & con2 & con3 & con4
        
        '''sell 조건 '''
#         con5=(df.loc[idx[0]:idx[-1],'buy'].shift(8)==True) #5일뒤 매도하는 조건
        df_copy.loc[idx[0]:idx[-1], 'sell'] = df_copy.loc[idx[0]:idx[-1],'buy'].shift(8)
    return df

In [4]:
# dbhandler = DB_Handle(db_name='kospi_fin')
dh = DataHandle(db_name='kospi_fin')
ec = EconomicIndicators()
bt = BackTesting()
ut = Utils()

In [5]:
# df = dh.get_data_for_pjt(period='all', companys=['포스코케미칼', '미래에셋생명', '현대로템', '한진칼'])
# df['종목명'] = df['코드'].apply(lambda x: dh.KOSPI_CODES[x])

In [6]:
df = pd.read_csv('../result.csv')
df['코드'] = df['코드'].apply(lambda x: str(x).zfill(6))

In [7]:
df_test = make_signal(df)

In [8]:
bt.MONEY = 5000000000

In [9]:
bt.buy_sell(df=df_test, buy='buy', sell='sell')

In [10]:
bt.update_information()

2018-02-07
2018-02-12
2018-02-13
2018-02-14
2018-02-19
2018-02-20
2018-02-21
2018-02-22
2018-02-23
2018-02-26
2018-02-27
2018-02-28
2018-03-02
2018-03-05
2018-03-06
2018-03-07
2018-03-08
2018-03-09
2018-03-12
2018-03-13
2018-03-14
2018-03-15
2018-03-16
2018-03-19
2018-03-20
2018-03-21
2018-03-22
2018-03-23
2018-03-26
2018-03-27
2018-03-28
2018-03-29
2018-03-30
2018-04-02
2018-04-03
2018-04-04
2018-04-05
2018-04-06
2018-04-09
2018-04-10
2018-04-11
2018-04-12
2018-04-13
2018-04-16
2018-04-17
2018-04-18
2018-04-19
2018-04-20
2018-04-23
2018-04-24
2018-04-25
2018-04-26
2018-04-27
2018-04-30
2018-05-02
2018-05-03
2018-05-04
2018-05-08
2018-05-09
2018-05-10
2018-05-11
2018-05-14
2018-05-15
2018-05-16
2018-05-17
2018-05-18
2018-05-21
2018-05-23
2018-05-24
2018-05-25
2018-05-28
2018-05-29
2018-05-30
2018-05-31
2018-06-01
2018-06-04
2018-06-05
2018-06-07
2018-06-08
2018-06-11
2018-06-12
2018-06-14
2018-06-15
2018-06-18
2018-06-19
2018-06-20
2018-06-21
2018-06-22
2018-06-25
2018-06-26
2018-06-27

2021-04-27
2021-04-28
2021-04-29
2021-04-30
2021-05-03
2021-05-04
2021-05-06
2021-05-07
2021-05-10
2021-05-11
2021-05-12
2021-05-13
2021-05-14
2021-05-17
2021-05-18
2021-05-20
2021-05-21
2021-05-24
2021-05-25
2021-05-26
2021-05-27
2021-05-28
2021-05-31
2021-06-01
2021-06-02
2021-06-03
2021-06-04
2021-06-07
2021-06-08
2021-06-09
2021-06-10
2021-06-11
2021-06-14
2021-06-15
2021-06-16
2021-06-17
2021-06-18
2021-06-21
2021-06-22
2021-06-23
2021-06-24
2021-06-25
2021-06-28
2021-06-29
2021-06-30
2021-07-01
2021-07-02
2021-07-05
2021-07-06
2021-07-07
2021-07-08
2021-07-09
2021-07-12
2021-07-13
2021-07-14
2021-07-15
2021-07-16
2021-07-19
2021-07-20
2021-07-21
2021-07-22
2021-07-23
2021-07-26
2021-07-27
2021-07-28
2021-07-29
2021-07-30
2021-08-02
2021-08-03
2021-08-04
2021-08-05
2021-08-06
2021-08-09
2021-08-10
2021-08-11
2021-08-12
2021-08-13
2021-08-17
2021-08-18
2021-08-19
2021-08-20
2021-08-23
2021-08-24
2021-08-25
2021-08-26
2021-08-27
2021-08-30
2021-08-31
2021-09-01
2021-09-02
2021-09-03

In [11]:
bt.JOUNAL

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
0,2018-02-07,089590,30131,3,127.0,매수
1,2018-02-07,014680,70000,3,295.0,매수
2,2018-02-07,008700,4720,3,20.0,매수
3,2018-02-12,000660,74500,3,314.0,매수
4,2018-02-12,003060,3444,3,15.0,매수
...,...,...,...,...,...,...
11826,2022-12-29,192820,74100,3,312.0,매도
11827,2022-12-29,039130,60500,3,255.0,매도
11828,2022-12-29,128940,298000,3,1256.0,매도
11829,2022-12-29,032830,71000,3,299.0,매도


In [21]:
bt.ECONO_INFORM[bt.ECONO_INFORM['실현손익'] > 0.0].shape[0] / bt.ECONO_INFORM.shape[0]

0.4056999161777033

In [14]:
bt.MONEY

4982278254.0

In [16]:
bt.ECONO_INFORM['실현손익'].sum()

-3561648.0

In [27]:
bt.JOUNAL[bt.JOUNAL['날짜'] == '2018-02-27']

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
82,2018-02-27,047810,51600,3,218.0,매도
83,2018-02-27,074610,15950,3,67.0,매도
84,2018-02-27,051910,398000,3,1678.0,매도
85,2018-02-27,042660,24950,3,105.0,매도
86,2018-02-27,000720,38091,3,161.0,매도
87,2018-02-27,009830,32568,3,137.0,매도
88,2018-02-27,000660,78700,3,332.0,매수
89,2018-02-27,005490,378500,3,1596.0,매도
90,2018-02-27,005490,378500,3,1596.0,매수


In [28]:
import plotly.express as px

In [29]:
px.line(x=bt.ECONO_INFORM['날짜'], y=bt.ECONO_INFORM['실현손익'])

In [15]:
bt.MONEY

205152.0

In [16]:
bt.JOUNAL[bt.JOUNAL['날짜'] == '2018-02-23']

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분


In [17]:
bt.JOUNAL[bt.JOUNAL['날짜'] == '2018-03-05']

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분


In [18]:
bt.JOUNAL[bt.JOUNAL['종목코드'] == '016360']

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
30,2018-03-22,016360,41550,3,175.0,매수
31,2018-03-22,016360,41550,3,175.0,매도
92,2018-05-11,016360,37150,3,157.0,매수
94,2018-05-11,016360,37150,3,157.0,매도
115,2018-05-29,016360,37550,3,158.0,매수
133,2018-06-11,016360,38200,3,161.0,매도


In [19]:
condi = (df_test['코드'] == '016360') & ((df_test['buy'] == True) | (df_test['sell'] == True))

In [20]:
df_test[condi]

,Unnamed: 0,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금,종목명,5_mom,10_mom,20_mom,buy,sell,구매수량,코드,상장일
644392,644392,2018-02-19,41300,41150,41850,41000,387594,16007381900,삼성증권,101.474201,94.616266,NaN,True,False,3,016360,1988-03-28
644400,644400,2018-03-02,39550,39000,39800,38600,246215,9641798800,삼성증권,98.751561,97.896040,90.711009,False,True,3,016360,1988-03-28
644406,644406,2018-03-12,41800,40600,41900,40250,424952,17472617000,삼성증권,108.150065,102.576687,102.702703,True,False,3,016360,1988-03-28
644414,644414,2018-03-22,41550,41150,42350,40900,469144,19623935450,삼성증권,102.592593,105.859873,101.589242,True,True,3,016360,1988-03-28
644422,644422,2018-04-03,38700,38950,39300,38350,402843,15562520550,삼성증권,98.724490,94.505495,98.348158,False,True,3,016360,1988-03-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645488,645488,2022-08-01,34550,35050,35050,34350,216699,7498084550,삼성증권,101.171303,104.538578,101.917404,False,True,3,016360,1988-03-28
645547,645547,2022-10-28,32000,31850,32350,31700,362420,11630059300,삼성증권,105.610561,100.946372,102.236422,True,False,3,016360,1988-03-28
645554,645554,2022-11-08,33550,32750,33550,32700,369256,12266623600,삼성증권,104.517134,110.000000,108.051530,True,False,3,016360,1988-03-28
645555,645555,2022-11-09,33650,33600,33800,33350,295850,9937236000,삼성증권,104.503106,110.327869,107.507987,False,True,3,016360,1988-03-28


In [21]:
bt.MONEY

205152.0

In [22]:
bt.ASSET

{'003000': {'평단가': 69006.0, '수량': 6},
 '025620': {'평단가': 286966, '수량': 3},
 '003780': {'평단가': 3305, '수량': 3},
 '001680': {'평단가': 26100, '수량': 3},
 '001470': {'평단가': 3745, '수량': 3},
 '011090': {'평단가': 2400, '수량': 3},
 '000490': {'평단가': 10800, '수량': 3},
 '213500': {'평단가': 16300, '수량': 3},
 '000080': {'평단가': 35800, '수량': 3},
 '042700': {'평단가': 16525, '수량': 3},
 '010050': {'평단가': 879, '수량': 3},
 '000320': {'평단가': 14800, '수량': 3},
 '214330': {'평단가': 2345, '수량': 3},
 '005070': {'평단가': 28150, '수량': 3},
 '130660': {'평단가': 13950, '수량': 3},
 '024110': {'평단가': 11450, '수량': 3},
 '000150': {'평단가': 90200, '수량': 3}}

In [23]:
bt.JOUNAL[bt.JOUNAL['종목코드'] == '006800']

,날짜,종목코드,체결단가,체결수량,매매비용,매매구분
53,2018-04-11,006800,9000,3,38.0,매수
72,2018-04-23,006800,9230,3,39.0,매도
116,2018-05-29,006800,9820,3,41.0,매수
134,2018-06-11,006800,9520,3,40.0,매도
382,2019-09-11,006800,7520,3,32.0,매수
392,2019-09-25,006800,7530,3,32.0,매도
660,2020-11-03,006800,8770,3,37.0,매수
662,2020-11-03,006800,8770,3,37.0,매도
755,2021-02-15,006800,9960,3,42.0,매수
760,2021-02-25,006800,9700,3,41.0,매도


In [24]:
bt.ECONO_INFORM.iloc[0:20, :]

,날짜,매수총합,매도총합,매매비용,실현손익
0,2018-02-12,233832,0.0,329.0,0.0
1,2018-02-13,2565,0.0,4.0,0.0
2,2018-02-22,1267857,0.0,1781.0,0.0


In [30]:
ut.making_mt_plot(handle_data=df_test, code='000990', jounal=bt.JOUNAL)

2018-02-14 2018-02-23
2018-03-09 2018-03-16
2019-04-30 2019-05-09
2019-07-01 2019-07-08
2020-01-17 2020-01-28
2020-02-13 2020-02-20
2020-04-07 2020-04-14
2020-04-21 2020-04-28
2020-05-07 2020-06-12
2020-08-13 2020-08-21
2020-09-08 2020-09-15
2020-11-03 2020-12-01
2021-02-16 2021-04-27
2021-07-12 2021-07-19
2021-09-06 2021-09-13
2021-10-15 2021-10-22
2021-12-16 2021-12-23
2022-03-16 2022-03-23
2022-04-14 2022-04-21
2022-05-12 2022-05-19
2022-08-16 2022-08-23
2022-08-31 2022-09-07
2022-10-14 2022-10-21
2022-10-26 2022-11-02


In [208]:
from copy import deepcopy
dates = []
buf = []
for _, row in bt.JOUNAL[bt.JOUNAL['종목코드'] == '010140'].iterrows():
    print(row['날짜'], row['매매구분'])
    if len(buf) == 2:
        duf_copy = deepcopy(buf)
        dates.append(duf_copy)
        print(dates)
        buf.clear()
        
    if not buf:
        if row['매매구분'] == '매수':
            buf.append(row['날짜'])
            print(buf)
    if row['매매구분'] == '매도':
        buf.append(row['날짜'])
        print(buf)
    

2018-03-08 매수
['2018-03-08']
2018-03-15 매도
['2018-03-08', '2018-03-15']
2018-05-24 매수
[['2018-03-08', '2018-03-15']]
['2018-05-24']
2018-05-31 매도
['2018-05-24', '2018-05-31']
2018-08-20 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31']]
['2018-08-20']
2018-08-27 매도
['2018-08-20', '2018-08-27']
2019-02-15 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31'], ['2018-08-20', '2018-08-27']]
['2019-02-15']
2019-02-21 매수
2019-02-22 매도
['2019-02-15', '2019-02-22']
2019-11-08 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31'], ['2018-08-20', '2018-08-27'], ['2019-02-15', '2019-02-22']]
['2019-11-08']
2019-11-15 매도
['2019-11-08', '2019-11-15']
2020-05-20 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31'], ['2018-08-20', '2018-08-27'], ['2019-02-15', '2019-02-22'], ['2019-11-08', '2019-11-15']]
['2020-05-20']
2020-05-27 매도
['2020-05-20', '2020-05-27']
2020-11-05 매수
[['2018-03-08', '2018-03-15'], ['2018-05-24', '2018-05-31'], ['2018-08-20', '2018-08-

In [149]:
a = list(bt.JOUNAL[(bt.JOUNAL['종목코드'] == '010140') & (bt.JOUNAL['매매구분'] == '매수')][['날짜', '매매구분']].itertuples(index=False, name=None))

In [150]:
b = list(bt.JOUNAL[(bt.JOUNAL['종목코드'] == '010140') & (bt.JOUNAL['매매구분'] == '매도')][['날짜', '매매구분']].itertuples(index=False, name=None))

In [151]:
# a = bt.JOUNAL[(bt.JOUNAL['종목코드'] == '034220') & (bt.JOUNAL['매매구분'] == '매수')][['날짜', '매매구분']].set_index('날짜').to_()['매매구분']
# b = bt.JOUNAL[(bt.JOUNAL['종목코드'] == '139130') & (bt.JOUNAL['매매구분'] == '매도')][['날짜', '매매구분']].set_index('날짜').to_dict()['매매구분']

In [12]:
start = pd.to_datetime('19000101')
end = pd.to_datetime('20230115')
code = '377740'

In [13]:
condition = (df['날짜'] > start) & (df['날짜'] < end) & (df['코드'] == code)

In [42]:
df[condition].max()['종가']

10650

In [4]:
sql = f'''select * from 제주항공 '''

In [5]:
res = dbhandler.fetch_base(sql)
res = datahandler(res)

In [6]:
res

,코드,날짜,종가,시가,최고가,최저가,누적거래량,누적거래대금
0,89590,2015-11-06,36785,37856,39385,35255,3144622,117995303950
1,89590,2015-11-09,34414,37244,37588,33994,459701,16175815800
2,89590,2015-11-10,32693,33649,35791,32693,341823,11608062150
3,89590,2015-11-11,34185,33114,34988,32885,361457,12348714700
4,89590,2015-11-12,34950,34873,35485,34147,207591,7280780950
...,...,...,...,...,...,...,...,...
1772,89590,2023-01-19,16400,16350,16600,15950,666623,10933975350
1773,89590,2023-01-20,16750,16350,16950,16100,661493,11004799900
1774,89590,2023-01-25,16190,16820,16900,15850,1137683,18475949590
1775,89590,2023-01-26,16070,16000,16300,15900,513798,8255311640


In [12]:
from pykrx import stock
stock.get_index_fundamental("20210101", "20210130", "1001")

,종가,등락률,PER,선행PER,PBR,배당수익률
날짜,,,,,,
2021-01-04,2944.45,2.47,30.200001,0.0,1.19,1.44
2021-01-05,2990.57,1.57,30.680000,0.0,1.20,1.42
2021-01-06,2968.21,-0.75,30.450001,0.0,1.20,1.43
2021-01-07,3031.68,2.14,31.110001,0.0,1.22,1.40
2021-01-08,3152.18,3.97,32.349998,0.0,1.27,1.35
2021-01-11,3148.45,-0.12,32.310001,0.0,1.27,1.35
2021-01-12,3125.95,-0.71,32.080002,0.0,1.26,1.36
2021-01-13,3148.29,0.71,32.310001,0.0,1.27,1.35
2021-01-14,3149.93,0.05,32.330002,0.0,1.27,1.35


In [ ]:
dfs = stock.get_index_fundamental(start,end,code)

In [13]:
import pandas as pd

In [16]:
dbhandler.DB_NAME[:-4]

'kospi'

In [ ]:
pd.DataFrame()